In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("S08").getOrCreate()

In [8]:
spark

In [40]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

#Cambiar estructura
schema = StructType([
    StructField("id_folio", StringType(), True),
    StructField("fecha_solicitud", StringType(), True),
    StructField("hora_solicitud", StringType(), True),
    StructField("tipo_de_entrada", StringType(), True),
    StructField("tema_solicitud", StringType(), True),
    StructField("sexo", StringType(), True),
    StructField("edad", StringType(), True),
    StructField("colonia_solicitud", StringType(), True),
    StructField("alcaldia_solicitud", StringType(), True),
    StructField("codigo_postal_solicitud", StringType(), True),
    StructField("estatus", StringType(), True),
    StructField("fecha_concluido", StringType(), True),
    StructField("alcaldia_catalogo", StringType(), True),
    StructField("colonia_catalogo", StringType(), True),
    StructField("longitud", StringType(), True),
    StructField("latitud", StringType(), True)
])

sepomex_df = spark.read.option("header","true")\
    .option("delimiter",",")\
    .option("skipRows","1")\
    .option("encoding","UTF-8")\
    .schema(schema)\
    .csv("sepomex.csv")

sepomex_df\
    .select(['codigo_postal_solicitud','colonia_solicitud','alcaldia_solicitud'])\
    .distinct()\
    .createOrReplaceTempView("sepomex")

locatel = spark.read.option("header","true")\
    .option("delimiter",",")\
    .option("skipRows","1")\
    .option("encoding","UTF-8")\
    .schema(schema)\
    .csv("locatel2024.csv")

locatel\
    .select(['codigo_postal_solicitud','colonia_solicitud','alcaldia_solicitud'])\
    .distinct()\
    .createOrReplaceTempView("locatel")

In [50]:
spark.sql("SHOW TABLES").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |  locatel|       true|
|         |  sepomex|       true|
+---------+---------+-----------+



In [ ]:
strsql="""
    SELECT COALESCE(s.colonia_solicitud,'TOTAL_C') AS ciudad,
           COALESCE(l.tema_solicitud, 'TOTAL_TS') AS tema_solicitud,
           COALESCE(l.sexo, 'TOTAL_S') AS sexo,
           COUNT(*) AS nr,
           GROUPING_ID(s.colonia_solicitud, l.tema_solicitud) AS grupo_id
    FROM locatel l
    JOIN sepomex s ON (l.codigo_postal_solicitud = s.codigo_postal_solicitud)
    GROUP BY CUBE(s.colonia_solicitud, l.tema_solicitud)
    ORDER BY 1, 2
"""

spark.sql(strsql).show(40, truncate=False)

AnalysisException: [MISSING_AGGREGATION] The non-aggregating expression "sexo" is based on columns which are not participating in the GROUP BY clause.
Add the columns or the expression to the GROUP BY, aggregate the expression, or use "any_value(sexo)" if you do not care which of the values within a group is returned.;
Sort [ciudad#1334 ASC NULLS FIRST, tema_solicitud#1335 ASC NULLS FIRST], true
+- Aggregate [colonia_solicitud#1343, tema_solicitud#1344, spark_grouping_id#1342L], [coalesce(colonia_solicitud#1343, TOTAL_C) AS ciudad#1334, coalesce(tema_solicitud#1344, TOTAL_TS) AS tema_solicitud#1335, coalesce(sexo#1122, TOTAL_S) AS sexo#1336, count(1) AS nr#1337L, spark_grouping_id#1342L AS grupo_id#1338L]
   +- Expand [[id_folio#1117, fecha_solicitud#1118, hora_solicitud#1119, tipo_de_entrada#1120, tema_solicitud#1121, sexo#1122, edad#1123, colonia_solicitud#1124, alcaldia_solicitud#1125, codigo_postal_solicitud#1126, estatus#1127, fecha_concluido#1128, alcaldia_catalogo#1129, colonia_

In [ ]:
strsql="""
    SELECT ciudad, tema_solicitud, nr
    FROM (
        SELECT COALESCE(s.colonia_solicitud,'TOTAL_C') AS ciudad,
           COALESCE(l.tema_solicitud, 'TOTAL_TS') AS tema_solicitud,
           COALESCE(l.sexo, 'TOTAL_S') AS sexo,
           COUNT(*) AS nr,
           GROUPING_ID(s.colonia_solicitud, l.tema_solicitud) AS grupo_id
        FROM locatel l
        JOIN sepomex s ON (l.codigo_postal_solicitud = s.codigo_postal_solicitud)
        GROUP BY CUBE(s.colonia_solicitud, l.tema_solicitud)
    ) x
    WHERE NOT (ciudad = 'TOTAL_C' AND tema_solicitud != 'TOTAL_TS)
    ORDER BY ciudad, grupo_id, tema_solicitud
"""

spark.sql(strsql).show(4000, truncate=False)

In [ ]:
strsql="""
    SELECT ciudad, tema_solicitud, nr
    FROM (
        SELECT COALESCE(s.colonia_solicitud,'TOTAL_C') AS ciudad,
           COALESCE(l.tema_solicitud, 'TOTAL_TS') AS tema_solicitud,
           COALESCE(l.sexo, 'TOTAL_S') AS sexo,
           COUNT(*) AS nr,
           GROUPING_ID(s.colonia_solicitud, l.tema_solicitud) AS grupo_id
        FROM locatel l
        JOIN sepomex s ON (l.codigo_postal_solicitud = s.codigo_postal_solicitud)
        WHERE s.colonia_solicitud IN('Acapulco de Juarez', 'Acayucan','Actopan')
        GROUP BY CUBE(s.colonia_solicitud, l.tema_solicitud)
    ) x
    WHERE NOT (ciudad = 'TOTAL_C' AND tema_solicitud != 'TOTAL_TS)
    AND NOT (ciudad= 'TOTAL_C' AND sexo != 'TOTAL_S')
    ORDER BY ciudad, grupo_id, tema_solicitud
"""

spark.sql(strsql).show(4000, truncate=False)